# Fuzzy Text Matching

## Prerequisites
- Install the Python package `thefuzz` in your favorite package manager.

Is Vienna the same as Wien, or Vienna, AT, Bécs? Vienna, austria. Wienna

In [6]:
def read_words(fname='/usr/share/dict/words'):
    with open(fname, 'rt') as f:
        words = f.readlines()
    return [word.strip() for word in words]

In [7]:
words = read_words()

In [8]:
len(words)

235886

In [9]:
words[0:5]

['A', 'a', 'aa', 'aal', 'aalii']

In [10]:
def find_word(words, word):
    for candidate in words:
        if candidate == word:
            return candidate
    return None

In [14]:
result = find_word(words, 'whiskye')

In [15]:
print(result)

None


In [16]:
from thefuzz import fuzz

## Distance metric of strings
Take x and y. Distance is 0 if x=y, increasing in some "natural" metric of distance.

### Levenshtein
How many characters do you have to add, delete or change in x to get y?

```
levenshtein('whisky', 'whiskey') = 1
levenshtein('whiskey', 'whiskye') = 2
```

In [17]:
fuzz.ratio('whisky', 'whiskey')

92

In [22]:
1 - 1 / (len('whisky') + len('whiskey')) 

0.9230769230769231

In [21]:
fuzz.ratio('app', 'apple')

75

In [23]:
def find_word_fuzzy(words, word):
    best_match = None
    best_score = 0
    for candidate in words:
        score = fuzz.ratio(candidate, word)
        if score > best_score:
            best_match = candidate
            best_score = score
    return best_match

In [25]:
result = find_word_fuzzy(words, 'whiskye')
print(result)

whisky


In [27]:
find_word_fuzzy(words, 'pálinka')

'lina'

In [29]:
%timeit find_word_fuzzy(words, 'whiskye')

87.2 ms ± 1.32 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


How much is 87ms? Say 6 million users, comparing agains 300k city names. How long will it take?
$$
6\times 10^6 \times 87 \times 10^{-3}
$$
For $N$ queries among $K$ words, this takes $\approx N\times K$ time.

## Binary search
All variants complete, on average, in $N\times\log(K)$ time
- binary search
- binary tree
- B tree

Most database engines index the data with B-trees.

## Hash tables
A Python dictionary is a hash table for keys. The key of the dict is converted into a "hash" by a cryptographic function. Lookup by key takes $\approx 1$ time. This can be unbelievably fast.

In [30]:
N = len(words)

In [32]:
words[N//2]

'modifier'

In [34]:
words[N//4]

'eagle'

In [35]:
word_list = words

In [36]:
word_dictionary = {k: '' for k in words}

In [37]:
len(word_list)

235886

In [38]:
len(word_dictionary)

235886

In [39]:
word_dictionary['apple']

''

In [40]:
%timeit 'whisky' in words

3.18 ms ± 1.9 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [41]:
%timeit 'whisky' in word_dictionary

25.6 ns ± 0.104 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


This is 125,000 times faster. You shouldn't care about speed of laptop, you should care about the algorithm.

In [48]:
letters = 'abcdefghijklmnopqrstuvwxyz'
words_by_first_letter = dict()
for word in words:
    first_letter = word[0].lower()
    if first_letter in words_by_first_letter:
        words_by_first_letter[first_letter].append(word)
    else:
        words_by_first_letter[first_letter] = [word]


In [50]:
len(words_by_first_letter['q'])

1152

In [51]:
words_by_first_letter

{'a': ['A',
  'a',
  'aa',
  'aal',
  'aalii',
  'aam',
  'Aani',
  'aardvark',
  'aardwolf',
  'Aaron',
  'Aaronic',
  'Aaronical',
  'Aaronite',
  'Aaronitic',
  'Aaru',
  'Ab',
  'aba',
  'Ababdeh',
  'Ababua',
  'abac',
  'abaca',
  'abacate',
  'abacay',
  'abacinate',
  'abacination',
  'abaciscus',
  'abacist',
  'aback',
  'abactinal',
  'abactinally',
  'abaction',
  'abactor',
  'abaculus',
  'abacus',
  'Abadite',
  'abaff',
  'abaft',
  'abaisance',
  'abaiser',
  'abaissed',
  'abalienate',
  'abalienation',
  'abalone',
  'Abama',
  'abampere',
  'abandon',
  'abandonable',
  'abandoned',
  'abandonedly',
  'abandonee',
  'abandoner',
  'abandonment',
  'Abanic',
  'Abantes',
  'abaptiston',
  'Abarambo',
  'Abaris',
  'abarthrosis',
  'abarticular',
  'abarticulation',
  'abas',
  'abase',
  'abased',
  'abasedly',
  'abasedness',
  'abasement',
  'abaser',
  'Abasgi',
  'abash',
  'abashed',
  'abashedly',
  'abashedness',
  'abashless',
  'abashlessly',
  'abashment',


In [54]:
def find_word_hash(dict_of_words, word):
    first_letter = word[0].lower()
    return find_word_fuzzy(dict_of_words[first_letter], word)

In [55]:
find_word_hash(words_by_first_letter, 'whiskye')

'whisky'

In [56]:
%timeit find_word_fuzzy(words, 'whiskye')

87.6 ms ± 945 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [57]:
%timeit find_word_hash(words_by_first_letter, 'whiskye')

1.46 ms ± 14.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [58]:
len(words_by_first_letter['w'])

3944

On average, speedup should be 26x, because there are 26 buckets in which to search.

## Applications
- Cities on GitHub
- Cities on CVs: https://github.com/ceumicrodata/ariadne/blob/main/ariadne/ariadne.py
- Company names
- Public agency names
- Person names

## Resources
- thefuzz
- unidecode can help with words like 'Сергей'

  
## Reading
- [Falsehoods about names](https://www.kalzumeus.com/2010/06/17/falsehoods-programmers-believe-about-names/)
- [Joel Spolsky on Unicode](https://www.joelonsoftware.com/2003/10/08/the-absolute-minimum-every-software-developer-absolutely-positively-must-know-about-unicode-and-character-sets-no-excuses/)
- [Introduction to Entity Resolution](https://dev.to/korenmiklos/wish-i-could-be-like-david-watts-2edp)